# Random City Model
_Felix Haumann_

Prerequisites:
- Excel file with city map where Roads are marked with "X"
- Excel map needs to have one lot marked with "D" which is the depot

Model parameters:
- Every road accessible lot will have a house
- House have between 1 and 15 households
- Households are assigned to the houses at (insert probability here)
- Every household can potentially receive parcels
- Probability for a household is (insert probability here)
- The probability is calculated for each household within a house and the resulting parcels then summed up for that house

Model limitations:
- Houses have no orientation so parcels can be delivered to the 'backside'

---
# SETUP

### Library imports

In [ ]:
import copy
import datetime
import math
import random
import os
from typing import Tuple, Dict, Any, List
import json

import xlwings as xw
import pandas as pd
import numpy as np
import time
import ast
import csv


from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder
from sklearn.cluster import KMeans
from IPython.display import display
import string
from pprint import pprint
from helper import generate_letter_combinations, save_df_as_csv, read_df_from_csv, elapsed_time, export_map_to_excel_with_formatting
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


## Helper functions

In [ ]:
def read_excel_map_as_df(wb_path, ws_name, col_range, skiprows, number_of_rows, number_of_cols):
    col_names = list(range(number_of_cols))
    print("read_excel_map_as_df:",col_range, skiprows, "number_of_rows", number_of_rows, "number_of_cols", number_of_cols)
    df = pd.read_excel(wb_path, sheet_name=ws_name, usecols=col_range, skiprows=skiprows, nrows=number_of_rows, names=col_names)
    print(df)
    return df

In [ ]:
def get_lot_xy_from_lot_name(ref_dict: dict, lot_name:str):
    return ref_dict[lot_name][key_h_xy]

In [ ]:
def write_master_house_dict_to_csv(data, filename):

    # Define the fieldnames for the CSV file
    fieldnames = ["lot",
                  key_h_xy,
                  key_h_lot_use,
                  key_h_households,
                  key_h_parcels,
                  key_h_delivery_district,
                  key_h_path_finding_val,
                  key_h_delivery_runs_required,
                  key_h_parcels_redirected_conv,
                  key_h_parcels_redirected_auto

                  # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                  ]

    # Write the dictionary to the CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for key, value in data.items():
            writer.writerow({"lot": key,
                             key_h_xy: value[key_h_xy],
                             key_h_lot_use: value[key_h_lot_use],
                             key_h_households: value[key_h_households],
                             key_h_parcels: value[key_h_parcels],
                             key_h_delivery_district: value[key_h_delivery_district],
                             key_h_path_finding_val: value[key_h_path_finding_val],
                             key_h_delivery_runs_required: value[key_h_delivery_runs_required],
                             key_h_parcels_redirected_conv: value[key_h_parcels_redirected_conv],
                             key_h_parcels_redirected_auto: value[key_h_parcels_redirected_auto]

                             # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                             })

def read_csv_to_master_house_dict(filename):
    temp_dict = {}

    # Read the CSV file and populate the dictionary
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            lot = row["lot"]
            xy = tuple(map(int, row[key_h_xy].strip("()").split(",")))
            lot_use = row[key_h_lot_use]
            households = int(row[key_h_households])
            parcels = int(row[key_h_parcels])
            delivery_district = row[key_h_delivery_district]
            path_finding_val = int(row[key_h_path_finding_val])
            delivery_runs_required = int(row[key_h_delivery_runs_required])
            parcels_redirected_conv = int(row[key_h_parcels_redirected_conv])
            parcels_redirected_auto = int(row[key_h_parcels_redirected_auto])

            # INSERT NEW ENTRIES ABOVE
            temp_dict[lot] = {key_h_xy: xy,
                              key_h_lot_use: lot_use,
                              key_h_households: households,
                              key_h_parcels: parcels,
                              key_h_delivery_district: delivery_district,
                              key_h_path_finding_val: path_finding_val,
                              key_h_delivery_runs_required: delivery_runs_required,
                              key_h_parcels_redirected_conv: parcels_redirected_conv,
                              key_h_parcels_redirected_auto: parcels_redirected_auto

                              # INSERT NEW ENTRIES ABOVE (MIND THE COMMA)
                              }
    return temp_dict

In [ ]:
def write_or_read_depots(depots_xy: list, path: str, read_or_write: str):
    if read_or_write == "w":
        # Writing to CSV file
        with open(path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(depots_xy)
    elif read_or_write == "r":
        # Reading from CSV file
        with open(path, mode='r') as file:
            reader = csv.reader(file)
            depots_xy_read = [(int(row[0]), int(row[1])) for row in reader]
        return depots_xy_read

In [ ]:
def get_excel_column_name(col_index):
    """
    Returns the Excel column name (e.g., 'A', 'B', ..., 'Z', 'AA', 'AB', ..., 'ZZ', 'AAA', ...)
    for a given column index.
    """
    col_name = ''
    while col_index > 0:
        col_index, remainder = divmod(col_index - 1, 26)
        col_name = chr(65 + remainder) + col_name
    return col_name

## Model parameters

### Paths

In [ ]:
# Excel files
blank_wb_city = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\CITY_BLANK.xlsx"
blank_ws_city = "BLANK"

model_output_path = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\model_output"

excel_pathvis_wb = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\PathVisualisation_TEST.xlsx"
excel_pathvis_ws = "VIS"

# CSV filenames
# TODO: add check if csv files are all available
csv_name_map_base = "map_base"
csv_distance_matrix_base = "distance_matrix_district_"

master_house_csv = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\csv_files\master_houses_dict.csv"
depots_xy_csv = r"C:\Users\fhaum\OneDrive\401 MASTER - Masterarbeit\04 Kalkulationen\pythonProject\csv_files\depots_xy.csv"



### Switches
rerun_simulation: decides whether the simulation will be run again or whether the available csv files will be used

In [ ]:
rerun_simulation = True
use_example_parameters = 2

### City parameters

In [ ]:
if use_example_parameters == 1:
    # Defining city boundaries
    num_of_rows = 30  # (X)
    num_of_cols = 40  # (Y)
    # Todo extract delivery districts from Excel map
    delivery_districts={"A":[(0,0),(19,4)],
                        "B":[(0,5),(19,9)],
                        "C":[(0,10),(19,14)],
                        "D":[(0,15),(19,19)],
                        "E":[(20,0),(39,4)],
                        "F":[(20,5),(39,9)],
                        "G":[(20,10),(39,14)],
                        "H":[(20,15),(39,19)],
                        "I":[(0,20),(39,29)]
                        }

elif use_example_parameters == 2:
    num_of_rows = 51
    num_of_cols = 50
    delivery_districts = {"A": [(0, 0), (19, 19)],
                      "B": [(20, 0), (39, 19)],
                      "C": [(0, 20), (19, 40)],
                      "D": [(20, 20), (39, 40)],
                      "E": [(0, 41), (49, 50)],
                      "F": [(40, 0), (49, 40)]
                      }

elif use_example_parameters == 3:
    num_of_rows = 100
    num_of_cols = 100
    delivery_districts = {"A": [(0, 0), (33, 23)],
                      "B": [(34, 0), (71, 26)],
                      "C": [(72, 0), (99, 35)],
                      "D": [(0, 24), (34, 47)],
                      "E": [(35, 27), (71, 52)],
                      "F": [(72, 36), (99, 66)],
                      "G": [(0, 48), (34, 71)],
                      "H": [(35, 53), (71, 75)],
                      "I": [(72, 67), (99, 99)],
                      "J": [(0, 72), (34, 99)],
                      "K": [(35, 76), (71, 99)]
                      }

else:
    raise ValueError("No parameters defined")

print(f"""
Using parameter {use_example_parameters}:
Map size = {num_of_cols, num_of_rows}
Delivery districts = {delivery_districts}
""")


### Map parameters

In [ ]:
# Defining the map boundaries of the Excel map
first_row = 3
first_col = 3

# Parameters for reading the Excel map
column_range = f"{get_excel_column_name(first_col)}:{get_excel_column_name(first_col+num_of_cols-1)}"
print(column_range)
# column_range = "C:AP"
skiprows = 1
map_char_street_read = "X"  # Map char which is read from the Excel template
map_char_depot = "D"
map_char_parcel = "P"
map_char_house = "H"
map_char_empty_lot = "."
map_char_street_write = " "  # Map char which will be printed to the visual outputs
name_string_depot = "DEPOT"





# # Defining the map boundaries from the Excel file
# first_row = 3
# first_col = 3
# last_row = 102
# last_col = 102
# num_of_rows = last_row-first_row+1
# num_of_cols = last_col-first_col+1
# column_range = "C:CX"
# skiprows = 1
# map_char_street = "X"
# map_char_depot = "D"
# name_string_depot = "DEPOT"
#
# # DEBUG Switch back on
# delivery_districts = {"A": [(0, 0), (33, 23)],
#                       "B": [(34, 0), (71, 26)],
#                       "C": [(72, 0), (99, 35)],
#                       "D": [(0, 24), (34, 47)],
#                       "E": [(35, 27), (71, 52)],
#                       "F": [(72, 36), (99, 66)],
#                       "G": [(0, 48), (34, 71)],
#                       "H": [(35, 53), (71, 75)],
#                       "I": [(72, 67), (99, 99)],
#                       "J": [(0, 72), (34, 99)],
#                       "K": [(35, 76), (71, 99)]
#                       }



# Pathfinding variables
# These variables are assigned to the cells for pathfinding
# Tiles with val 1 = walkable
# Tiles with val >1 = walkable but higher cost
# Tiles with val 0 or negative = not walkable
# Note: the last tile (end_coordinates) must have a positive value otherwise it cannot be reached
path_val_depot = 200
path_val_house = 500
path_val_house_with_parcels = 100
path_val_empty_lot = 0
path_val_road = 1

### Multiplier parameters

In [ ]:
par_sys_distance_multiplier = 0.5  # Modifying the distances
par_sys_meters_per_tile = 10  # Defining how big the size of a lot is in meters

### System parameters

#### Conventional delivery

In [ ]:
par_sys_conv_type_name = "conventional"
par_sys_conv_vehicle_speed = 10

#### Autonomous delivery

In [ ]:
par_sys_aut_type_name = "autonomous"
par_sys_aut_vehicle_capacity = 1
par_sys_aut_vehicle_speed = 6

### Weights & distributions

In [ ]:
# Parcel quantity choices and weights
parcel_quantity_choices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
parcel_quantity_weights = [0.8996, 0.0634, 0.0233, 0.0085, 0.0031, 0.0011, 0.0004, 0.0001, 0.0001, 0.0001, 0.0001]

# Household size choices and weights
household_size_choices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
household_size_weights = [0.4, 0.35, 0.045, 0.04, 0.035, 0.03, 0.025, 0.02, 0.015, 0.01, 0.005, 0.0025, 0.0125, 0.005, 0.005]

# Parcel delivery redirection rate
parcel_redirected_choices = [False, True]
parcel_redirected_weights = {par_sys_conv_type_name: [0.95, 0.05], par_sys_aut_type_name: [0.97, 0.03]}

---
# Central datastructures

## Master_Houses_Dict

In [ ]:
master_houses_dict = {}

# IMPORTANT: CHANGES TO DICT STRUCTURE REQUIRE INTEGRATION INTO IMPORT AND EXPORT FUNCTION (write_master_house_dict_to_csv)
# IMPORTANT: ALSO ADD TO POPULATE MASTER HOUSES DICT FUNCTION
# IMPORTANT: ALSO ADD TO OUTPUT DICT (master_districts_output_dict)

key_h_xy = "xy"
key_h_lot_use = "lot_use"
key_h_path_finding_val = "path_finding_val"
key_h_households = "households"
key_h_parcels = "parcels"
key_h_delivery_district = "delivery_district"
key_h_delivery_runs_required = "delivery_runs_required"  # Depending on carry capacity autonomous vehicles might have to drive to house more often
key_h_parcels_redirected_conv = "parcels_redirected_conv"  # Parcels which could not be delivered and were returned to the depot
key_h_parcels_redirected_auto = "parcels_redirected_auto"  # Parcels which could not be delivered and were returned to the depot

# Final format will be: {"AAA":{"xy":(0,0), "lot_use":"P", "households":2, "parcels":4, "deliver_district":"A", "delivery_runs_required":4}, "AAB":{"xy":(1,0), "lot_use": ".", "households":0, "parcels":0, "delivery_district":"", "delivery_runs_required":0, "redirected_parcels":1}}

In [ ]:
def populate_master_houses_dict(height:int = num_of_rows, width:int = num_of_cols):
    letter_combinations = generate_letter_combinations()
    for y in range(width):
        for x in range(height):
            name = next(letter_combinations)
            master_houses_dict[name] = {key_h_xy: (x, y), key_h_lot_use: None, key_h_households: 0, key_h_parcels: 0, key_h_delivery_district: "", key_h_path_finding_val:0, key_h_delivery_runs_required:0, key_h_parcels_redirected_conv:0, key_h_parcels_redirected_auto:0}

populate_master_houses_dict()

In [ ]:
pprint(master_houses_dict)

## Master_Routes_Dict

In [ ]:
# Dict that will contain all the information about the conventional delivery districts

# Final format will be:
# {<districtA>: {"paths":{<vehicle_id1>: ['DEPOT1', 'ATI', 'AXC', 'BMM']}, "path_lengths" : {<vehicle_id1>: 450}},
# <districtB>: {"paths":{<vehicle_id1>: ['DEPOT1', 'ATI', 'AXC', 'BMM']}, "path_lengths" : {<vehicle_id1>: 450}}}

master_routes_dict = {}

key_r_paths = "paths"
key_r_path_lengths = "path_lengths"


## Master_Autonomous_Routes_Dict

In [ ]:
# Dict that will contain all the information about the autonomous delivery systems

# Final format will be:
# {<districtA>: {"distance": 1550, "further_parameter": "something"}},
# <districtB>:{"distance": 2540, "further_parameter": "something_else"}}

master_autonomous_dict = {}

key_a_distance = "distance"

## Master_Model_Output_Dict

In [ ]:
# Dict that will contain all the return information of the model

# Final format will be:
# {<districtA>: {"parcels": 180, "parcels_redirected": 4, "stops": 110, "distance_conventional": 853, "distance_autonomous":1759, "vehicles_used_autonomous":15}},
# <districtB>: {"parcels": 158, "parcels_redirected": 1, "stops": 120, "distance_conventional": 965, "distance_autonomous":2864, "vehicles_used_autonomous":15}}

master_districts_output_dict = {}

key_d_parcels = "parcels"
key_d_stops = "stops"
key_d_drop_factor = "drop_factor"
key_d_distance_conventional = "dist_conv"  # Distance for route-based conventional delivery in meters
key_d_distance_autonomous = "dist_auto"  # Distance for milk-run based autonomous delivery in meters
key_d_vehicles_autonomous = "vehicles_auto"
key_d_parcels_redirected_conv = "parcels_redirected_conv"
key_d_parcels_redirected_auto = "parcels_redirected_auto"


## Master_City_Dict

In [ ]:
# Dict that will contain all the general information about the city

# Final format will be:
# {"houses": 567, "households":1759, "total_parcels":156, "total_parcels_redirected": 14, "total_stops": 110}

master_city_output_dict = {}

key_c_houses = "houses"
key_c_households = "households"
key_c_total_parcels = "total_parcels"
key_c_total_parcels_redirected = "total_parcels_redirected"
key_c_total_stops = "total_stops"
key_c_city_dimensions = "city_dimensions"

---
# Prepare maps

### Base map from Excel

In [ ]:
if rerun_simulation:
    # DEBUG
    # map_base = read_excel_map_as_df(wb_path=blank_wb_city, ws_name=blank_ws_city, col_range=column_range,
    #                                 skiprows=skiprows, number_of_rows=num_of_rows, number_of_cols=num_of_cols)
    map_base = read_excel_map_as_df(wb_path=blank_wb_city, ws_name=blank_ws_city, col_range=column_range,
                                    skiprows=skiprows, number_of_rows=num_of_rows, number_of_cols=num_of_cols)
    save_df_as_csv(map_base, csv_name_map_base)
else:
    map_base = read_df_from_csv(csv_name_map_base)

pd.set_option('display.max_columns', None, 'display.max_rows', None)
display(map_base)



### Copy base map

In [ ]:
map_lot_use = map_base.copy()
map_lot_use = map_lot_use.fillna(0)  # Replace NaN with empty lots

In [ ]:
map_parcel = map_base.copy()

In [ ]:
map_pathfinding = map_base.copy()

In [ ]:
map_lot_names = map_base.copy()

### Map styling

In [ ]:
def display_styled_map(map_to_style):
    center_style = [
        {'selector': 'td',
         'props': [('text-align', 'center')]}
    ]
    styled_map = map_to_style.style
    styled_map.set_table_styles(center_style)
    display(styled_map)

---
# Populate city with houses

In [ ]:
def lot_next_to_road(map_df: pd.DataFrame, x:int, y:int, map_char_street) -> bool:
    """
    Checks if current lot is next to a road
    """
    # OG version
    # own = map_df.iat[y, x]
    # above = map_df.iat[y - 1, x] if y - 1 >= 0 else 0
    # left = map_df.iat[y, x - 1] if x - 1 >= 0 else 0
    # below = map_df.iat[y + 1, x] if y + 1 < map_df.shape[0] else 0
    # right = map_df.iat[y, x + 1] if x + 1 < map_df.shape[1] else 0

    own = map_df.iat[x, y]
    above = map_df.iat[x - 1, y] if x - 1 >= 0 else 0
    left = map_df.iat[x, y - 1] if y - 1 >= 0 else 0
    below = map_df.iat[x + 1, y] if x + 1 < map_df.shape[0] else 0
    right = map_df.iat[x, y + 1] if y + 1 < map_df.shape[1] else 0



    #print("own",own, "above",above, "left", left, "right", right, "below",below)  # Debug


    around = {"a": True if above == map_char_street else False,
              "l": True if left == map_char_street else False,
              "r": True if right == map_char_street else False,
              "b": True if below == map_char_street else False}
    any_surround = any(around.values())

    if own != map_char_street and any_surround:
        return True
    else:
        return False


In [ ]:
def select_household_size():
    result = random.choices(household_size_choices, household_size_weights, k=1)
    res_int = result[0]
    return res_int

In [ ]:
def select_parcel_quantity():
    result = random.choices(parcel_quantity_choices, parcel_quantity_weights, k=1)
    res_int = result[0]
    return res_int

In [ ]:
def parcels_were_redirected(system_type: str):
    """
    Returns True if parcels for household were redirected else False
    :param system_type: "conventional" | "autonomous"
    :return:
    """
    result = random.choices(parcel_redirected_choices, parcel_redirected_weights[system_type], k=1)
    res_bool = result[0]
    return res_bool

In [ ]:
pprint(master_houses_dict)

In [ ]:
def evaluate_lot_use(map_df: pd.DataFrame) -> list:
    """
    Evaluates every lot in the master_dict for it use.
    If a lot is by a road a house will be places, the households in the house chosen at random based on the choices and weights defined.
    The parcels each household receives which are again based in choices and weights are summed up for each house.
    The master_dict will be edited to incorporate the lot use for printing a pretty map.
    The coordinates of the depot or depots will be returned as list of tuples.
    :param map_df:
    :return: depot_coordinates
    """
    depot_coordinates = []
    i = 0
    for lot, info in master_houses_dict.items():
        print(f"i = {i}")
        i+=1
        x, y = info[key_h_xy]
        print("X", x, "y", y)
        # print("lot", lot)
        cell_val = map_df.iat[x,y] # DEBUG war im og [y,x]
        print(f"{lot}[{y},{x}]{cell_val}, [{x},{y}]{map_df.iat[x, y]}")
        # print(cell_val)
        if cell_val == 0:
            if lot_next_to_road(map_df, x, y, map_char_street_read):
                households_in_house = select_household_size()
                parcels_to_house = 0
                parcels_to_house_redirected = 0
                for household in range(households_in_house):
                    parcels_for_household = select_parcel_quantity()
                    if parcels_for_household > 0 and parcels_were_redirected(par_sys_conv_type_name):
                        parcels_to_house_redirected += parcels_for_household
                        print(f"Redirected {parcels_for_household} parcels. Total: {parcels_to_house_redirected} parcels redirected")
                    parcels_to_house += parcels_for_household
                info[key_h_households] = households_in_house
                info[key_h_parcels] = parcels_to_house
                info[key_h_parcels_redirected_conv] = parcels_to_house_redirected
                if parcels_to_house > 0:
                    info[key_h_lot_use] = map_char_parcel
                    info[key_h_path_finding_val] = path_val_house_with_parcels
                else:
                    info[key_h_lot_use] = map_char_house
                    info[key_h_path_finding_val] = path_val_house
            else:
                info[key_h_lot_use] = map_char_empty_lot
        elif cell_val == map_char_depot:
            info[key_h_lot_use] = map_char_depot
            info[key_h_path_finding_val] = path_val_depot
            depot_coordinates.append(info[key_h_xy])
        elif cell_val == map_char_street_read:
            info[key_h_lot_use] = map_char_street_write
            info[key_h_path_finding_val] = path_val_road
        else:
            info[key_h_lot_use] = "WTF"
    return depot_coordinates


In [ ]:
if rerun_simulation:
    depots_xy = evaluate_lot_use(map_lot_use)
    write_or_read_depots(depots_xy, depots_xy_csv, "w")
    write_master_house_dict_to_csv(master_houses_dict, master_house_csv)
    pprint(master_houses_dict)
    print(f"Depot locations: {depots_xy}")
    print("------")
    print("Lot use evaluated")
else:
    print("Reading lot use from csv...")
    depots_xy = write_or_read_depots([], depots_xy_csv, "r")
    master_houses_dict = read_csv_to_master_house_dict(master_house_csv)
    pprint(master_houses_dict)
    print(f"Depot locations: {depots_xy}")
    print("------")
    print("Lot use read from csv")


In [ ]:
pprint(master_houses_dict)


---
# Populate maps

## Show parcel locations on map

In [ ]:
def populate_parcel_map():
    for name, info in master_houses_dict.items():
        x,y = info[key_h_xy]
        lot_use = info[key_h_lot_use]
        map_parcel.iat[x, y] = lot_use #DEBUG was [y,x]

In [ ]:
populate_parcel_map()
display_styled_map(map_parcel)

## Populate lot names map

In [ ]:
def populate_lot_names_map():
    for name, info in master_houses_dict.items():
        x,y = info[key_h_xy]
        lot_use = info[key_h_lot_use]
        print_name = name if lot_use not in [".", " "] else ""
        lot_use = lot_use if lot_use in ["."] else ""
        map_lot_names.iat[x, y] = f"{print_name}{lot_use}"  # DEBUG was [y,x]

In [ ]:
populate_lot_names_map()
display_styled_map(map_lot_names)

---
# Pathfinding

## Setup Pathfinding

#### Transform map for pathfinding

In [ ]:
map_pathfinding = map_pathfinding.replace(map_char_street_read, -999)  # Replace 'X' values with -999 --> Road
map_pathfinding = map_pathfinding.replace(map_char_depot, -1000)  # Replace 'D' values with -1000 --> Depot
map_pathfinding = map_pathfinding.fillna(path_val_empty_lot) # Replace NaN values with -2 --> Empty lots
map_pathfinding = map_pathfinding.apply(pd.to_numeric, errors='coerce', downcast='integer')  # Transform float values to ints
map_pathfinding = map_pathfinding.mask(map_pathfinding >= 0, -1)  # Replace positive numeric values with -1
map_pathfinding = map_pathfinding.replace(-999, path_val_road)  # Replace -999 values with road_val
map_pathfinding = map_pathfinding.replace(-1000, path_val_depot)  # Replace -1000 values with depot_val

map_pathfinding

### Populate pathfinding map with values from master_dict

In [ ]:
def populate_pathfinding_map():
    for lot, info in master_houses_dict.items():
        x, y = info[key_h_xy]
        path_value = info[key_h_path_finding_val]
        map_pathfinding.iat[x,y] = path_value  # Debug was [y,x]

In [ ]:
populate_pathfinding_map()
display(map_pathfinding)

In [ ]:
def instantiate_grid(map_df: pd.DataFrame, export_matrix:bool = False):
    pathfinding_map_matrix = map_df.values.tolist()
    grid_export = Grid(matrix=pathfinding_map_matrix)
    if export_matrix:
        return grid_export, pathfinding_map_matrix
    else:
        return grid_export

In [ ]:
def find_path(grid: Grid, start_coords: tuple, end_coords: tuple):
    """
    Finds path on a provided grid between two nodes

    :param grid:
    :param start_coords:
    :param end_coords:
    :return: path_res, path_len_res
    """
    start_y, start_x  = start_coords  # Debug Was start_x, start_y
    end_y, end_x  = end_coords  # Debug was end_x, end_y
    start = grid.node(start_x, start_y)
    end = grid.node(end_x, end_y)
    finder = AStarFinder(diagonal_movement=DiagonalMovement.never)
    path_res, runs = finder.find_path(start, end, grid)
    path_len_res = len(path_res)
    # print('operations:', runs, 'path length:', path_len)
    # print(grid.grid_str(path=path, start=start, end=end))
    grid.cleanup()
    return path_res, path_len_res

In [ ]:
def draw_path_to_map(path_to_draw: list, map_to_print: pd.DataFrame):
    path_marker = "🟥"
    print_map = map_to_print.copy()
    # FixMe: if first or last node is above or below the start or finish pop it of because it will otherwise overlap the stop or start
    #path_to_draw.pop(0)
    #path_to_draw.pop(len(path_to_draw)-1)
    for loc in path_to_draw:
        y, x = loc  # Debug was x,y # no idea why it is flipped here
        # print(loc)
        print_map.iat[x,y] = path_marker    # Debug was [y,x]
    return print_map

In [ ]:
def draw_from_lot_to_lot(start_lot: str|tuple, end_lot: str|tuple):
    if type(start_lot) is str:
        start_lot = master_houses_dict[start_lot][key_h_xy]
    if type(end_lot) is str:
        end_lot = master_houses_dict[end_lot][key_h_xy]

    path, path_len = find_path(map_pathfinding_grid, start_lot, end_lot)
    display_styled_map(draw_path_to_map(path, map_parcel))


## Execute pathfinding

In [ ]:
map_pathfinding_grid, map_pathfinding_matrix = instantiate_grid(map_pathfinding, export_matrix=True)

In [ ]:
# Draw path from depot to random parcel location for testing
def choose_far_away_node():
    master_houses_dict_as_list = list(master_houses_dict.items())
    # Iterate in reverse using reversed() and a for loop
    for lot_name, info in reversed(master_houses_dict_as_list):
        if info[key_h_lot_use] == map_char_parcel:
            # print(f"Found {lot_name} with {info[key_h_xy]}")
            return tuple(info[key_h_xy])

debug_coord_s = (1,1)
debug_coord_e = choose_far_away_node()

path, path_len = find_path(map_pathfinding_grid, debug_coord_s, debug_coord_e)
display_styled_map(draw_path_to_map(path, map_parcel))


---
# Create district routes

## Split parcel locations into delivery districts

In [ ]:
def determine_delivery_district(coords):
    """
    Evaluates provided coordinates to return the delivery district it belongs to
    :param coords:
    :return:
    """
    y, x = coords # Debug was x, y
    for district, coords_range in delivery_districts.items():
        x_min, y_min = coords_range[0]
        x_max, y_max = coords_range[1]
        if x_min <= x <= x_max and y_min <= y <= y_max:
            return district
    raise ValueError

In [ ]:
def evaluate_lots_for_delivery_district():
    """
    Checks which lot with a parcel quantity of > 0 belongs into which delivery district
    """
    for lot, info in master_houses_dict.items():
        print(lot, info)
        if info[key_h_parcels]>0:
            info[key_h_delivery_district] = determine_delivery_district(info[key_h_xy])

In [ ]:
evaluate_lots_for_delivery_district()

In [ ]:
pprint(master_houses_dict)

In [ ]:
def split_parcel_locations_into_districts() -> dict:
    """
    Splits all nodes that have parcels into a dict with the delivery district as key.
    The depot(s) are added to each district
    :return: {"A":{"AAB":(1,3),"AAD":(4,5)}, "B":{"DAF":(8,4),....}}
    """
    district_quantities = {}
    for district in delivery_districts.keys():
        district_nodes = {}
        for index, depot_coordinates in enumerate(depots_xy):
            depot_name = f"{name_string_depot}{index+1}"
            district_nodes[depot_name] = depot_coordinates
        for lot, info in master_houses_dict.items():
            if info[key_h_delivery_district] == district:
                district_nodes[lot] = info[key_h_xy]
        district_quantities[district] = district_nodes
    return district_quantities

In [ ]:
nodes_per_district = split_parcel_locations_into_districts()

In [ ]:
# DEBUG
for node in nodes_per_district["A"]:
    if name_string_depot not in node:
        print(master_houses_dict[node])

In [ ]:
pprint(nodes_per_district)

## Generate distance matrices within districts

In [ ]:
def instantiate_distance_matrices(nodes_in_district: dict, importing_matrices: bool = False) -> dict:
    """
    Creates individual (blank) distance matrices for all delivery districts.
    :param nodes_in_district:
    :param importing_matrices: Flag whether the blank distance matrices will be generated.
    :return: distance_matrices_dict
    {"A": <DF with all lots with parcels in district A>,
    "B": <DF with all lots with parcels in district A>}
    """
    distance_matrices_dict = {}
    if not importing_matrices:
        for district, nodes in nodes_in_district.items():
            lot_names = list(nodes.keys())
            distance_matrix = pd.DataFrame(index=lot_names, columns=lot_names)
            distance_matrices_dict[district] = distance_matrix
    return distance_matrices_dict

In [ ]:
instantiate_distance_matrices(nodes_per_district)

In [ ]:
def get_remaining_stops(stops: dict, cur_key:str) -> dict:
    """
    Evaluates the remaining nodes which the distance has not been calculated for yet for the current key.
    :param stops:
    :param cur_key:
    :return:
    """
    remaining_stops = {}
    found_key = False
    for key, value in stops.items():
        if key == cur_key:
            found_key = True
        elif found_key:
            remaining_stops[key] = value
    return remaining_stops

In [ ]:
def populate_distance_matrix(nodes_in_district: dict, dst_matrix: pd.DataFrame, district: str) -> pd.DataFrame:
    """
    Calculates the distances between each node
    :param dict nodes_in_district: Dict of all lots that will get parcels {'LHU': (38, 76), 'LHZ': (43, 76), 'LIF': (49, 76)}
    :param pd.DataFrame dst_matrix: DF of the instantiated distance matrix
    :param district
    :return:
    """
    return_df = dst_matrix
    for lot_a, node_a in nodes_in_district.items():
        print("------")
        remaining_stops = get_remaining_stops(stops=nodes_in_district, cur_key=lot_a).items()
        remaining_nodes = len(dict(remaining_stops).keys())
        print(f"Remaining nodes: {remaining_nodes}")
        print(f"Checking distances for: {lot_a}{node_a}")
        for lot_b, node_b in remaining_stops:
            _, tiles = find_path(grid=map_pathfinding_grid,start_coords=node_a,end_coords=node_b)
            #print(f"[{district}] Distance from {lot_a}{node_a} to {lot_b}{node_b} = {distance} units")
            return_df.loc[lot_a, lot_b] = tiles * par_sys_meters_per_tile  # DEBUG was just = tiles
            return_df.loc[lot_b, lot_a] = tiles * par_sys_meters_per_tile

    return_df = return_df.fillna(0)  # Fill distance between itself with 0
    return return_df

In [ ]:
print(nodes_per_district)

In [ ]:
# Populate distance matrices
if rerun_simulation:
    start_time = time.time()
    distance_matrices = instantiate_distance_matrices(nodes_in_district=nodes_per_district, importing_matrices=False)
    for district in delivery_districts.keys():
        print(f"Current district: {district}")
        tmp_df = populate_distance_matrix(nodes_per_district[district], distance_matrices[district], district)
        distance_matrices[district] = tmp_df
        print(tmp_df)
        save_file_name = f"{csv_distance_matrix_base}{district}"
        save_df_as_csv(df_to_save=tmp_df, csv_name_no_filetype=save_file_name, with_header_and_rows=True)
        print(f"Saved '{save_file_name}'.")
    pprint(distance_matrices)
    elapsed_time(start_time)
else:

    distance_matrices = instantiate_distance_matrices(nodes_in_district=nodes_per_district, importing_matrices=False)
    pprint(distance_matrices)
    column_names_dict = {}
    for district, distance_matrix in distance_matrices.items():
        column_names_dict[district] = distance_matrix.columns.tolist()

    #print("column_names_dict",column_names_dict)

    for district in delivery_districts.keys():
        #print(f"Column Names for district {district}: {column_names_dict[district]}")
        renamed_columns = {}
        for index, lot_name in enumerate(column_names_dict[district]):
            renamed_columns[index] = lot_name
        print(renamed_columns)
        print(f"Retrieving data from csv for district {district}")
        tmp_df = read_df_from_csv(f"{csv_distance_matrix_base}{district}", with_header_and_rows=True)
        #tmp_df = tmp_df.rename(columns=renamed_columns)
        distance_matrices[district] = tmp_df
        del tmp_df


    pprint(distance_matrices["A"]) #Debug

In [ ]:
display(distance_matrices)

## Create delivery routes

### Setup data model

In [ ]:
vrp_data = {}
vrp_data_key_distance_matrix = "distance_matrix"
vrp_data_key_num_vehicles = "num_vehicles"
vrp_data_key_depot = "depot"

In [ ]:
def prepare_data_model(distance_matrix: dict, number_of_vehicles: int = 1, depot_index: int = 0) -> dict:
    """
    Returns dictionary with distance matrix as np array, number of vehicles for the route calculating, and the index of the depot in the np array
    """
    vrp_data = {vrp_data_key_distance_matrix: distance_matrix.to_numpy(),
                vrp_data_key_num_vehicles: number_of_vehicles,
                vrp_data_key_depot: depot_index}
    return vrp_data


### Prepare data output

In [ ]:
def output_solution(data, manager, routing, solution):
    """
    Prints solution to console.
    :param data:
    :param manager:
    :param routing:
    :param solution:
    :return:
    """
    #print(f"Objective: {solution.ObjectiveValue()}")
    route_paths = {}
    route_distances = {}
    optimal_route_per_vehicle = {}
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        print_routes = ""
        route_distance = 0
        optimal_route = []
        while not routing.IsEnd(index):
            route_index = manager.IndexToNode(index)
            print_routes += f" {route_index} -> "
            optimal_route.append(route_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        print_routes += f"{manager.IndexToNode(index)}"
        optimal_route.append(manager.IndexToNode(index))  # Add index of depot as last node
        route_distances[vehicle_id] = route_distance
        route_paths[vehicle_id] = print_routes
        optimal_route_per_vehicle[vehicle_id] = optimal_route
    return route_paths, optimal_route_per_vehicle, route_distances

In [ ]:
def extract_lot_names_from_route_indices(route_indices_per_vehicle: dict, distance_matrix: pd.DataFrame):
    lot_names = distance_matrix.columns.tolist()
    all_routes_with_lot_names_per_vehicle = {}
    for vehicle_id, route in route_indices_per_vehicle.items():
        route_with_lot_names = []
        for node_index in route:
            route_with_lot_names.append(lot_names[node_index])
        all_routes_with_lot_names_per_vehicle[vehicle_id]=route_with_lot_names
    return all_routes_with_lot_names_per_vehicle

### Instantiate model

In [ ]:
def evaluate_vrp(distance_matrix: dict, number_of_vehicles: int = 1) -> tuple[dict, dict, dict]:
    """
    Evaluates the vrp for one delivery district for each set delivery vehicle.
    :param distance_matrix:
    :param number_of_vehicles:
    :return: print_routes, route_indices, path_distances
    """
    # Prepare data model
    data = prepare_data_model(distance_matrix=distance_matrix,
                              number_of_vehicles=number_of_vehicles,
                              depot_index=0)
    # Setup index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'],
                                           data['depot'])
    # Instantiate routing model
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Calculate cost of each path
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add distance constraint
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        5000,  # vehicle maximum travel distance
        True,  # start cumulate to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    print_route_per_vehicle = {}
    route_indices_per_vehicle = {}
    path_distance_per_vehicle={}
    # Print solution on console.
    if solution:
        print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle = output_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

    return print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle


### Execute model for each district

In [ ]:
def find_routes_for_delivery_districts(delivery_districts: dict, distance_matrices: dict):
    if rerun_simulation:
        print("Analysing delivery routes")
        routes_per_district = {}
        for district in delivery_districts:
            print(f"Analyzing delivery district: {district}")
            relevant_distance_matrix = distance_matrices[district]
            print_route_per_vehicle, route_indices_per_vehicle, path_distance_per_vehicle = evaluate_vrp(relevant_distance_matrix)
            routes_per_vehicle = extract_lot_names_from_route_indices(route_indices_per_vehicle, relevant_distance_matrix)
            for vehicle_id in routes_per_vehicle:
                print(f"Optimized route for vehicle {vehicle_id} in district {district}:")
                print(" -> ".join(routes_per_vehicle[vehicle_id]))
                print(f"Total distance for vehicle {vehicle_id} in district {district}: {path_distance_per_vehicle[vehicle_id]} units")
                print(f"----------")
            routes_per_district[district] = {}
            routes_per_district[district]["paths"] = routes_per_vehicle
            routes_per_district[district]["path_lengths"] = path_distance_per_vehicle
        print("Saving delivery routes to csv")
        df_routes_per_district = pd.DataFrame(routes_per_district)
        save_df_as_csv(df_routes_per_district, "vehicle_delivery_routes", True)

    else:
        print("Reading delivery routes from csv files")
        df_routes_per_district = read_df_from_csv("vehicle_delivery_routes", True)
    return df_routes_per_district


In [ ]:
routes_per_district = find_routes_for_delivery_districts(delivery_districts=delivery_districts, distance_matrices=distance_matrices)

### Populate master_routes_dict
<a id='sheesh'></a>

In [ ]:
def populate_master_dict_routes(df):
    """Populates the master_dict_route"""
    result = {}
    for index, row in df.iterrows():
        paths = row['paths']
        path_lengths = row['path_lengths']
        if isinstance(paths, str):
            paths = ast.literal_eval(paths)
        if isinstance(path_lengths, str):
            path_lengths = ast.literal_eval(path_lengths)
        result[index] = {
            'paths': paths,
            'path_lengths': path_lengths
        }
    return result

print(type(routes_per_district))

temp_ = routes_per_district.copy()
temp_ = temp_.T # Transpose df
pprint(temp_)
master_routes_dict = populate_master_dict_routes(temp_)
del temp_
print(master_routes_dict)

#print(master_dict_routes)

print(master_routes_dict["A"]["paths"][0])


## Visualize delivery route

In [ ]:
export_map_to_excel_with_formatting(map_parcel, excel_pathvis_wb, excel_pathvis_ws)

In [ ]:
# TODO visualizes

# Create milkruns for autonomous delivery

## Define sum function

Every point has to be visited once.
The delivery vehicle has to go from the depot to the point and return (depending on the carry capacity)
Distance per house = <distance_from_depot_to_node> * 2

In [ ]:
def calculate_runs_per_house(lot_name, carry_capacity: int = 1):
    parcel_qty = master_houses_dict[lot_name][key_h_parcels]
    runs = math.ceil(parcel_qty / carry_capacity)
    master_houses_dict[lot_name][key_h_delivery_runs_required] = runs
    return runs


In [ ]:
def calculate_milk_run_distances(df: pd.DataFrame, district_name, carry_capacity:int = 1):
    """
    Calculate the sum of all values in row 0 of a DataFrame.
    :param df:
    :return:
    """
    headers = list(df)
    # Filter Depots from list
    filtered_headers = [col for col in headers if name_string_depot not in col]
    row = df.iloc[0]
    row_values = row.tolist()
    #print(filtered_headers)
    # Get quantities for each lot

    path_length = 0
    for index, lot_name in enumerate(headers):
        if lot_name in filtered_headers:
            path_length_per_lot = 0
            run_multiplier = calculate_runs_per_house(lot_name, carry_capacity)
            distance = row_values[index] * par_sys_meters_per_tile  # Debug was only row_values[index]
            distance_both_ways = distance * 2
            distance_all_runs = distance_both_ways * run_multiplier
            path_length_per_lot += distance_all_runs
            print(f"[{district_name}]Path length for {lot_name}= {path_length_per_lot} Parcels={master_houses_dict[lot_name][key_h_parcels]}, Distance from Depot={row_values[index]}")
            path_length += path_length_per_lot

            # Calculate how many parcels were redirected
            parcels_to_house = master_houses_dict[lot_name][key_h_parcels]
            print(parcels_to_house)
            for parcel in range(parcels_to_house):
                redirected_parcel = parcels_were_redirected(par_sys_aut_type_name)
                master_houses_dict[lot_name][key_h_parcels_redirected_auto] += redirected_parcel if redirected_parcel > 0 else 0

    #print(row.tolist())
    return path_length


In [ ]:
print(distance_matrices["A"])
print(calculate_milk_run_distances(distance_matrices["A"], "A"))

In [ ]:
def populate_autonomous_delivery_dict():
    for district, distance_matrix in distance_matrices.items():
        master_autonomous_dict[district] = {}
        print(f"Calculating milk run distances for district {district}")
        distance = calculate_milk_run_distances(df=distance_matrix, district_name=district, carry_capacity=par_sys_aut_vehicle_capacity)
        master_autonomous_dict[district][key_a_distance] = distance
    print(master_autonomous_dict)


In [ ]:
populate_autonomous_delivery_dict()

# Model output

## Modifier layer
Apply defined multipliers to adjust parameters to approximate real-world values

In [ ]:
# Copy all master dicts to not modify those
mod_master_houses_dict = copy.deepcopy(master_houses_dict)
mod_master_city_dict = copy.deepcopy(master_city_output_dict)
mod_master_routes_dict = copy.deepcopy(master_routes_dict)
mod_master_autonomous_dict = copy.deepcopy(master_autonomous_dict)

# Modify all distances
print("Applying modifier for distances")
for district in delivery_districts:
    # Modify conventional distances
    current_route_path_length = master_routes_dict[district][key_r_path_lengths][0]  # Todo add possibility to change vehicles
    modified_route_path_length = round(current_route_path_length * par_sys_distance_multiplier)
    print(f"District {district} (conventional) distance was: {current_route_path_length} | modified to: {modified_route_path_length}")
    mod_master_routes_dict[district][key_r_path_lengths][0] = modified_route_path_length

    # Modify autonomous distances
    current_autonomous_path_length = master_autonomous_dict[district][key_a_distance]
    modified_autonomous_path_length = round(current_autonomous_path_length * par_sys_distance_multiplier)
    print(f"District {district} (autonomous) distance was: {current_autonomous_path_length} | modified to: {modified_autonomous_path_length}")
    mod_master_autonomous_dict[district][key_a_distance] = modified_autonomous_path_length



## Gather output

In [ ]:
def populate_master_city_output_dict():
    number_of_houses = 0
    number_of_households = 0
    number_of_parcels = 0
    number_of_redirected_parcels = 0
    number_of_stops = 0

    for lot, info in master_houses_dict.items():
        if info[key_h_lot_use] == map_char_house:
            number_of_houses += 1
        if info[key_h_lot_use] == map_char_parcel:
            number_of_stops += 1
        number_of_households += info[key_h_households]
        number_of_parcels += info[key_h_parcels]
        number_of_redirected_parcels += info[key_h_parcels_redirected_conv]

    master_city_output_dict[key_c_houses] = number_of_houses
    master_city_output_dict[key_c_households] = number_of_households
    master_city_output_dict[key_c_total_parcels] = number_of_parcels
    master_city_output_dict[key_c_total_stops] = number_of_stops
    master_city_output_dict[key_c_total_parcels_redirected] = number_of_redirected_parcels
    master_city_output_dict[key_c_city_dimensions] = (num_of_cols * par_sys_meters_per_tile, num_of_rows * par_sys_meters_per_tile)

In [ ]:
def populate_master_districts_output_dict():
    parcels_by_district = {}
    for district in delivery_districts:
        master_districts_output_dict[district] = {}
        total_parcels_in_district = 0
        total_stops_in_district = 0
        total_redirected_parcels_in_district_conv = 0
        total_redirected_parcels_in_district_auto = 0

        # Sum total number of stops in district
        for lot, info in master_houses_dict.items():
            # print("lot", lot, "info",info)
            if info[key_h_parcels] > 0 and info[key_h_delivery_district]==district:
                total_parcels_in_district += int(info[key_h_parcels])
                total_stops_in_district += 1

        # Sum total redirected parcels in district
        for lot, info in master_houses_dict.items():
            redirected_parcels_conv = info[key_h_parcels_redirected_conv]
            if redirected_parcels_conv  > 0 and info[key_h_delivery_district]==district:
                total_redirected_parcels_in_district_conv += redirected_parcels_conv

            redirected_parcels_auto = info[key_h_parcels_redirected_auto]
            if redirected_parcels_auto  > 0 and info[key_h_delivery_district]==district:
                total_redirected_parcels_in_district_auto += redirected_parcels_auto

        master_districts_output_dict[district][key_d_parcels] = total_parcels_in_district
        master_districts_output_dict[district][key_d_parcels_redirected_conv] = total_redirected_parcels_in_district_conv
        master_districts_output_dict[district][key_d_parcels_redirected_auto] = total_redirected_parcels_in_district_auto
        master_districts_output_dict[district][key_d_stops] = total_stops_in_district
        master_districts_output_dict[district][key_d_drop_factor] = 0 if total_stops_in_district == 0 else (total_parcels_in_district / total_stops_in_district)
        master_districts_output_dict[district][key_d_distance_conventional] = master_routes_dict[district][key_r_path_lengths][0]
        master_districts_output_dict[district][key_d_distance_autonomous] = master_autonomous_dict[district][key_a_distance]


## Model output

In [ ]:
populate_master_districts_output_dict()
populate_master_city_output_dict()

print("District data:")
pprint(master_districts_output_dict)

print("------")

print("City data:")
pprint(master_city_output_dict)

### Export dict to csv file

In [ ]:
def transform_output_dict_to_2d_shape(input_dict, join_key):
    """
    Transforms a dictionary into a 2D shape with join_key key-value pairs.
    """
    input_dict = copy.deepcopy(input_dict)

    return_list = []
    for key, val in input_dict.items():
        val[join_key] = key
        return_list.append(val)
    header = return_list[0].keys()
    return return_list, header

In [ ]:
not_yet_exported = True

In [ ]:
if not_yet_exported:
    now = datetime.datetime.now()
    formatted_timestamp = now.strftime("%Y-%m-%dT%H:%M:%S")
    master_city_output_dict_with_timestamp = copy.deepcopy(master_city_output_dict)

    master_city_output_dict_with_timestamp["timestamp"] = f"{formatted_timestamp}"

    master_districts_output_dict_with_timestamp = copy.deepcopy(master_districts_output_dict)
    for district, info in master_districts_output_dict_with_timestamp.items():
        master_districts_output_dict_with_timestamp[district]["timestamp"] = f"{formatted_timestamp}"

    model_output_metadata_name = f"Model_Output_City_Metadata.csv"
    model_output_full_path_metadata = f"{model_output_path}\\{model_output_metadata_name}"
    field_names_metadata = list(master_city_output_dict_with_timestamp)

    model_output_districts_name = f"Model_Output_City_Districts.csv"
    model_output_full_path_districts = f"{model_output_path}\\{model_output_districts_name}"


    if os.path.exists(model_output_full_path_metadata):
        write_header_metadata = False  # Set to False if file exists
    else:
        write_header_metadata = True   # Set to True if file does not exist

    with open(model_output_full_path_metadata, "a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=field_names_metadata)
        if write_header_metadata:
            writer.writeheader()
        writer.writerow(master_city_output_dict_with_timestamp)


    if os.path.exists(model_output_full_path_districts):
        write_header_districts = False  # Set to False if file exists
    else:
        write_header_districts = True   # Set to True if file does not exist

    with open(model_output_full_path_districts, 'a', newline='') as f:
        master_districts_output_as_list, header = transform_output_dict_to_2d_shape(master_districts_output_dict_with_timestamp,"district")
        writer = csv.DictWriter(f, fieldnames=header)
        if write_header_districts:
            writer.writeheader()
        # Write each dictionary as a separate row in the CSV file
        for item in master_districts_output_as_list:
            writer.writerow(item)
    #
    # with open(model_output_full_path_districts, "a", newline="") as f:
    #     writer = csv.DictWriter(f, fieldnames=field_names_districts)
    #     if write_header_districts:
    #         writer.writeheader()
    #     writer.writerow(master_districts_output_dict_with_timestamp)

    print("Exported csv files successfully")
    not_yet_exported = False
else:
    print("Current model run has already been exported. Rerun full model to prevent duplicates.")

In [ ]:
pprint(master_houses_dict)